In [1]:
!mkdir postgres-data
!docker run --rm \
  -e POSTGRES_PASSWORD=postgres \
  -p 54320:5432 \
  -v `pwd`/postgres-data:/var/lib/postgresql/data \
  pgvector/pgvector:pg17

Unable to find image 'pgvector/pgvector:pg17' locally
^C


In [2]:
!poetry add cohere

Using version ^5.13.5 for cohere

Updating dependencies
Resolving dependencies... (4.3s)Resolving dependencies... (4.0s)

Package operations: 10 installs, 0 updates, 0 removals

  - Installing filelock (3.16.1): Pending...
  - Installing fsspec (2024.12.0): Pending...
  - Installing pyyaml (6.0.2): Pending...
  - Installing pyyaml (6.0.2): Pending...
  - Installing fsspec (2024.12.0): Downloading... 0%
  - Installing pyyaml (6.0.2): Pending...
  - Installing fsspec (2024.12.0): Downloading... 0%
  - Installing pyyaml (6.0.2): Pending...
  - Installing filelock (3.16.1): Downloading... 0%
  - Installing fsspec (2024.12.0): Downloading... 0%
  - Installing pyyaml (6.0.2): Pending...
  - Installing fsspec (2024.12.0): Downloading... 0%
  - Installing pyyaml (6.0.2): Pending...
  - Installing filelock (3.16.1): Downloading... 100%
  - Installing fsspec (2024.12.0): Downloading... 0%
  - Installing pyyaml (6.0.2): Pending...
  - Installing fsspec (2024.12.0): Downloading... 0%
  - Installin

In [ ]:
from pydantic_ai import Agent, RunContext
from pydantic_ai.models import KnownModelName
from dataclasses import dataclass
from dotenv import load_dotenv
from typing import Any, List, cast
import os
from cohere import Client
import pydantic_core
import httpx

load_dotenv()

True

In [ ]:
DOCS_JSON = (
    'https://gist.githubusercontent.com/'
    'samuelcolvin/4b5bb9bb163b1122ff17e29e48c10992/raw/'
    '80c5925c42f1442c24963aaf5eb1a324d47afe95/logfire_docs.json'
)

In [ ]:
def build_search_db():
    client = http_client()

In [46]:
@dataclass
class Deps:
    cohere_embed: Client

In [75]:
# agent = Agent(cast(KnownModelName, os.getenv('PYDANTIC_AI_MODEL')), deps_type=Deps,
#             )
agent = Agent('gemini-1.5-pro', deps_type=Deps)

In [ ]:
@agent.tool
async def retrieve(context: RunContext[Deps], search_query: str) -> List[float]:
    """Retrieve documentation sections based on a search query.
    Args:
        context: The call context.
        search_query: The search query string.

    Returns:
        A list of float embeddings.
    """
    model = "embed-english-v3.0"
    input_type = "search_query"
    
    # Generate embeddings using the Cohere client
    res = context.deps.cohere_embed.embed(
        texts=[search_query],
        model=model,
        input_type=input_type,
        embedding_types=["float"]
    )
    
    embedding = res.embeddings.float
    embedding_json = pydantic_core.to_json(embedding).decode()
    
    rows = await context.deps.pool.fetch(
        'SELECT url, title, content FROM doc_sections ORDER BY embedding <-> $1 LIMIT 8',
        embedding_json,
    )
    return '\n\n'.join(
        f'# {row["title"]}\nDocumentation URL:{row["url"]}\n\n{row["content"]}\n'
        for row in rows
    )

<module>:8: No type or annotation for returned value 1


In [79]:
cohere_embed = Client()
deps = Deps(cohere_embed=cohere_embed)

In [80]:
q = 'How do I configure logfire to work with FastAPI?'
result = await agent.run(q, deps=deps)

In [81]:
result.data

'To configure Logfire with FastAPI, you can add the Logfire handler to your FastAPI logging setup.  Here\'s a basic example:\n\n```python\nimport logging\nfrom logfire import LogfireHandler\nfrom fastapi import FastAPI\n\n# Initialize FastAPI\napp = FastAPI()\n\n# Configure logging\nlogger = logging.getLogger(__name__)\nlogger.setLevel(logging.INFO)\n\n# Replace with your Logfire API key\nlogfire_handler = LogfireHandler("<YOUR_LOGFIRE_API_KEY>")\n\nlogger.addHandler(logfire_handler)\n\n# Example route\n@app.get("/")\nasync def root():\n    logger.info("Logfire message from FastAPI")\n    return {"message": "Hello World"}\n\n```\n\nRemember to replace `<YOUR_LOGFIRE_API_KEY>` with your actual Logfire API key.  This setup will send logs from your FastAPI application to Logfire.  You can customize the logging level, format, and other settings as needed.  Refer to the Logfire and FastAPI documentation for more advanced configuration options.\n'